# Intro to chemistRy

Welcome to the *chemistRy*, a product of the JMU Department of Chemistry and Biochemistry. This is the fourth lesson in the series which  aims to teach you about data visualization, data tidying, statistics, and a bit of R coding. If you don't know how to code, don't worry! These lessons assume no prior knowledge of code or R.

A few things to start:

1.   These lessons only work in Google Chrome
2.   If you want to save your progress, go to File> Save a Copy in Drive; then locate a spot in your Drive folder
3.   This is a special Google Colab for R, regular Colab runs in Python. If you want to make your own R notebook, follow this [link](https://colab.research.google.com/drive/16u6W4rl-AHX_yXlQx7HcgjAuNimfqwj_).

If you have questions, feel free to contact Dr. Chris Berndsen in the JMU Chemistry Department.


In [ ]:
#@title Run this block always
#@markdown This code will install the necessary packages
#install.packages("modeldata", quietly = TRUE)
install.packages("R.utils", quietly = TRUE)

## load packages
library(tidyverse)
#library(modeldata)
library(R.utils)

In [ ]:
#@title Run this block to download the data
#@markdown This code will import the data and process it into the data frame format. The CSV files are available to download in the file folder on the left side if you want to work on this project offline.
download.file("https://raw.githubusercontent.com/CEBerndsen/r_colab_data/main/guinier.csv", "guinier.csv")
download.file("https://raw.githubusercontent.com/CEBerndsen/r_colab_data/main/porod.csv", "porod.csv")
g <- read.csv("guinier.csv")
p <- read.csv("porod.csv")

gdf <- as_tibble(g)
pdf <- as_tibble(p)

# Data cleaning
Data comes in many shapes, sizes, and levels of complexity. Thus far we have worked with mostly tidy data, meaning it is formatted nicely for R and requires no changes to visualize. However, this is rarely the case with data we collect. We may need to organize data, remove columns or rows that are unnecessary for the analysis, or do calculations to get values with the right units. For small data sets, this is straightfoward to do manually in a spreadsheet program. However for complex or large data sets, this task becomes much more difficult to do efficiently by hand.

Take a look at the data set below:

In [ ]:
dim(gdf)
head(gdf)


This data set has 7200 rows of observations for 11 different values. In addition, but not shown, there is a second data set that needs to be paired with this one row by row. This is BIG DATA!

Here are the tasks we need to accomplish:


*   Combine the two data frames `gdf` and `pdf` and match up the datasets
*   Remove the path from the filename column
*   Remove the duplicate columns and some unnecessary data columns
*   Calculate the estimated $D_{max}$ value

Once we have the cleaned data set, we can summarize and visualize the data.



# Joining data frames
When you want to combine observations on the same data set from two sources, it is easy to think that you could just copy and paste the two spreadsheets and be done. But that act assumes the data are in the same order so that you can match the rows. This is not always the case especially with complex data sets. In R, we can use the `join` function to match and combine the two data frames together.

This [page](https://tavareshugo.github.io/r-intro-tidyverse-gapminder/08-joins/index.html) shows a nice visual of the different types of joins that are possible. We will be performing an `inner_join`, which means only rows that exist in both dataframes will be in the combined data frame.

To join data frames, you need to have a common column to match in both data sets. In our case  `filename`, `frame_number`, `well_num`, or `buffer` are common between the data sets. The `filename` is the ***most specific*** and the easiest choice. We could use a combination of `frame_number`, `well_num`, and `buffer` and that is probably the tidier choice.

`inner_join` takes 3 parameters: dataframe 1, dataframe 2, and `by=` which is the column name.



In [ ]:
# cdf is the new dataframe reference
# gdf and pdf are the two dataframes we are joining by filename
cdf <- inner_join(gdf, pdf, by = "filename")

# see the first 6 rows of the new data frame
head(cdf)
dim(cdf)

Notice the dimensions of the dataframe listed below the table. The new dataframe has the same number of observations just for more variables. You may see some of the columns have a .x or .y in the name, that is because they were duplicated in the join. While filename was also in common it was the name we matched, so it does not get a .x or .y suffix. We can eliminate some of these duplicates by joining another way.

In [ ]:
# use well_num, buffer, and frame_number to match up the data
# HINT: by= typically takes only a single value, how do we combine multiple values in a single name?
# refer to 001_r_intro if you forgot
cdf2 <- inner_join(gdf, pdf, by = )

head(cdf2)
dim(cdf2)

If you were successful in making `cdf2` then you should see a table that looks similar to the first join, but the dimensions are different. There are fewer columns but the same number of rows. Also, frame_number lacks the .x suffix, because we used that column along with couple more to join the data. Only filename has the .x.

What if you modify the code to include `filename` along with `frame_number`, `well_num`, or `buffer`? Try this!


# Cleaning the data
We now have joined the datasets and we did some cleaning inadvertantly when we used multiple factors to join the dataframes as this removed duplicate columns. But there is still more to do! The new dataframe has some extra columns and the observations in the filename column include the path. Now we will use the `separate` command to break about the filename column and the `select` commmand to remove extra columns. Finally, we will remove `NA` values using the filter command. We can do this step-wise and in one step and will learn both.

## Separating data in columns
The `separate` command splits observations in a column by some character or value. In our data, the filenames have the path which includes `\` between the directories, so we will split using that value.

In [ ]:
# need to provide dataframe name, column name, and the separator, and the new column names.
# add the dataframe name and column name removing the _____
# also name the new columns A, B, C, filename, remember you need to combine multiple names into a single assignment
cdf3 <- separate(cdf2, _______, sep = "\\\\",  into = _______))
head(cdf3)

Success! We split the column, now we need to remove columns A, B, and C along with a few others.

**Note:** You might have noticed that the `sep` command was filled in already, that's because `\` is treated unusually in R and needs to be "escaped". Typically we escape a special character with `\\` (ex. `\\.` or `\\?` but since we are literally wanting a `\` we need to escape it twice, thus the `\\\\`.

## Selecting columns
The command for selecting columns is `select` and like separate it requires a dataframe and the columns to keep. These can be indicated by names or by column number if known.



In [ ]:
# We want to keep filename, frame_number, well_num, buffer, rg_med, mass_vp, mass_vc
# put them in the space indicated
# no need for quotes around column names in this case.
cdf4 <- select(cdf3, ______)

head(cdf4)

In [ ]:
# by column number starting from left at 1
cdf5 <- select(cdf3, 5:9, 14, 21)

head(cdf5)

Using column numbers is easier than typing all that text, so use that if at all possible.

# Removing rows

Finally, we will remove the rows that contain `NA` values using the `filter` command. These values will affect the calculations and summarizing the data that comes next. However, sometimes they can be informative so filter NA values selectively. You can also filter based on specific values like names (ex. buffer name) or certain limits (ex. rg_med > 10).

Let's remove the `NA` values and make sure our rg_med values are all positive. `Filter` requires a dataframe and some sort of TRUE/FALSE statement like `buffer == "Buffer1"`. In this example if the value in buffer is Buffer1, then the statement is evaluated as TRUE and the row is kept, while if it is FALSE, then the entire row is discarded.

In [ ]:
# Add in the true\false statement saying rg_med is greater than 0
cdf6 <- filter(cdf5, _______)

# remove the na values
# R has an is.na(), but we need the opposite of that so we add an ! in front of the command
cdf7 <- filter(cdf6, !is.na(rg_med))

dim(cdf7)

# Doing calculations and adding columns
Sometimes we want to do additional processing like multiplying numbers together or correcting factors. We can do this using the `mutate` function. In this step, we write an equation to show how we want R to do the math. Here we will estimate the Dmax value by multiplying the median Rg by 3.

In [ ]:
# fill in the equation to multiply rg_med by 3
cdf8 <- mutate(cdf7, d_est = _____)

head(cdf8)

We did the cleaning in several different steps, but we could do it all in one set of code if we used "pipes". The symbol for a pipe is `%>%` or `|>` and allows you to feed the output of one function into another. See the piped version of our previous work below.

In [ ]:
cdf9 <- cdf2 %>%
  separate(., filename, sep = "\\\\",  into = c("A", "B", "C", "filename")) %>%
  select(., 5:9, 14, 21) %>%
  filter(rg_med > 0) %>%
  mutate(., d_est = rg_med*3)

head(cdf9)
dim(cdf9)


See how much easier that was! We did not have to keep track of a bunch of intermediate dataframe steps and we got the same answer.

Now that we have a clean data set, we can move onto summarizing the data both visually and statistically.




---

# Summarizing data

Big data sets are fun to visualize, but sometimes we need the numbers and statistics to help form conclusions. The current data set is >7000 rows long and composed of 30-50 measurements (`frame_number`) of several unique combinations of samples (`well number`) and buffer (`buffer`). To try to summarize this manually would be difficult and tedious. Fortunately, R offer a few methods to summarize the data. The first is a simple summary of the data frame which works well for homogenous data.


In [ ]:
# summary of the cdf9
summary(cdf9)

This command produced the basic statistics for the observations in each column but these statistics are meaningless because we have observations from a lot of different samples. To be useful, we want the statistics for all fifty frame numbers for each well number and buffer combination. To do this we are going to use a pipe along with two new functions `group_by` and `summarize` to tell R how to divide the data and then calculate the statistics. Like Excel, R has built in functions for `mean` and standard deviation (`sd`) so we can use those inside the summarize function.


In [ ]:
summary_df <- cdf9 %>%
  # fill in the column names that we want to group by with a comma between the names. There are 2 columns names needed here
  group_by(.,________) %>%
  summarize(mean_rg = mean(rg_med),
            sd_rg = sd(rg_med),
            # fill in the equations to calculate the mean and sd of Vp, add comma and a line break each time
            )

head(summary_df)

The summarized data file is greatly condensed and *hopefully* shows the mean and standard deviation for the desired value for each well and buffer combination. The group_by and summarize combination is an incredibly useful tool in data analysis.



---
Now you try with two new sets of data. Like the examples above, these are analysis of small angle X-ray scattering data and the resulting stats. However, they are from two sources, one an internal analysis and the other from the [SASBDB](https://www.sasbdb.org/), the international repository for SAXS data. We want to compare the internal analysis to the deposited numbers to determine the validity of the new analysis. This will require a few cleaning steps, joining, the data, and then calculation of the differences between experimental and reference values. The specific tasks are listed below:


1.   Separate the filename column in exp removing the filepath and .dat leaving only the id value (ex. SASDA26)

2.   Convert the Rg value in db from nanometers to Angstroms (rg_A)

3.   Join the exp and db data frames by the id value

4.   Reduce the combined dataframe by keeping only rg_ts, rg_A, mass_vp, mass, N (from exp) and N (from db)

5.   Determine the differences between rg_ts and rg_A, mass_vp and mass, and N and N.

6.   Calculate the mean, median, sd, and IQR for each difference (does not require the group_by %>% summary path)

7.   Plot distribution of differences in ggplot2 for each comparison and show the stats on the plot using an `annotate` layer.

##Good luck!






In [ ]:
#@title load the data
download.file("https://raw.githubusercontent.com/CEBerndsen/r_colab_data/main/SASBDB_analysis.csv", "exp.csv")
download.file("https://raw.githubusercontent.com/CEBerndsen/r_colab_data/main/SASBDB_protein_stats.csv", "db.csv")
e <- read.csv("exp.csv")
p <- read.csv("db.csv")

exp <- as_tibble(e)
db <- as_tibble(p)

In [ ]:
# show the layout of the experimental analysis
head(exp)

In [ ]:
# show the layout of the database
head(db)

In [ ]:
# Your turn!